In [3]:
%matplotlib inline
import numpy as np
import pandas as pd

import xgboost as xgb
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder

In [4]:
X_train = pd.read_pickle('X_train.pickle')
y_train = pd.read_pickle('y_train.pickle')
X_test = pd.read_pickle('X_test.pickle')
y_test = pd.read_pickle('y_test.pickle')

enc = LabelEncoder()
enc.fit(X_test['mail_category'].append(X_train['mail_category']))
X_train['mail_category'] = enc.transform(X_train['mail_category'])
X_test['mail_category'] = enc.transform(X_test['mail_category'])

enc.fit(X_test['mail_type'].append(X_train['mail_type']))
X_train['mail_type'] = enc.transform(X_train['mail_type'])
X_test['mail_type'] = enc.transform(X_test['mail_type'])

enc.fit(X_test['mail_id'].append(X_train['mail_id']))
X_train['mail_id'] = enc.transform(X_train['mail_id'])
X_test['mail_id'] = enc.transform(X_test['mail_id'])

In [28]:
dtrain = xgb.DMatrix(X_train.as_matrix(), label=y_train.astype(int).as_matrix())

In [58]:
params = {
    "objective": "reg:logistic",
    "booster" : "gbtree",
    "eval_metric": "auc",
    "eta": 0.05,
    "max_depth": 10,
    "subsample": 0.8,
    "colsample_bytree": 0.7
}

bm = xgb.train(params, dtrain, 30, verbose_eval=True)
prob = bm.predict(xgb.DMatrix(X_train.as_matrix()))

In [61]:
prob = bm.predict(xgb.DMatrix(X_test.as_matrix()))

In [85]:
cutoff = 0.24
ground = (y_test == 1)
pred = (prob > cutoff)
TP = (pred & ground).sum()
FP = (pred & ~ground).sum()
TN = (~pred & ~ground).sum()
FN = (~pred & ground).sum()
P = TP / (TP + FP)
R = TP / (TP + FN)
S = (2 * P * R) / (P + R)
Rat = (FP + FN) / TP
print((ground == pred).mean(), TP, FP, TN, FN, P, R, Rat, S)

0.939502300293 451 1536 44476 1357 0.226975339708 0.249446902655 6.41463414634 0.23768115942


In [35]:
bm.predict(xgb.DMatrix(X_test.as_matrix()))

array([ 0.18678927,  0.18966077,  0.1959596 , ...,  0.19055007,
        0.20995593,  0.19440895], dtype=float32)

In [49]:
y_test

435675    False
319657    False
2613      False
360369    False
457864    False
467648    False
321105    False
374296    False
453780    False
178569    False
431586    False
363160    False
297298    False
433654    False
112378    False
422629    False
289395    False
183402    False
436756    False
125040    False
286837    False
285060    False
160088    False
445122    False
2614      False
425914    False
447950    False
100797    False
60770     False
468544    False
          ...  
340600    False
472593    False
244147    False
453972    False
255018    False
212038    False
475319    False
482347     True
328163    False
467031    False
371523    False
484341    False
475587    False
266189    False
469626    False
134009    False
483714    False
473413    False
60415     False
405089    False
477242    False
253952    False
454610    False
477742    False
252083    False
102772    False
474979    False
476947    False
36630     False
178494     True
Name: clicked, dtype: bo

In [75]:
X_train

,sent_time,last_online,hacker_created_at,hacker_timezone,contest_login_count,contest_login_count_1_days,contest_login_count_1_days,contest_login_count_1_days,contest_login_count_1_days,contest_login_count_30_days,...,submissions_count_master_30_days,submissions_count_master_365_days,submissions_count_master_7_days,mail_category,mail_type,rcounts,urcounts,ocounts,uocounts,mail_id
98637,1455280999,1.455190e+09,1447248771,18000.0,1,0,0,0,0,0,...,4,37,3,0,0,0,0,0,0,51
86546,1455281000,1.455191e+09,1444305615,28800.0,1,0,0,0,0,0,...,1,58,1,0,0,0,0,0,0,126
251159,1455281000,1.455192e+09,1423128151,18000.0,3,0,0,0,0,1,...,4,5,4,0,0,0,0,0,0,133
86547,1455281008,1.455193e+09,1444742421,10800.0,1,0,0,0,0,0,...,0,4,0,0,0,0,0,0,0,42
68455,1455281013,1.455192e+09,1440755089,18000.0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,126
155565,1455281019,1.455192e+09,1383607184,-14400.0,1,0,0,0,0,0,...,50,50,1,0,0,0,0,0,0,126
280178,1455281019,1.455194e+09,1451802397,-25200.0,10,0,0,0,0,4,...,5,15,3,0,0,0,0,0,0,133
198687,1455281021,1.455191e+09,1407866817,7200.0,7,0,0,0,0,0,...,7,91,0,0,0,0,0,0,0,42
149744,1455281023,1.455192e+09,1353778104,25200.0,1,0,0,0,0,0,...,12,83,0,0,0,0,0,0,0,126
291145,1455281026,1.455192e+09,1452525985,18000.0,1,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,42


In [74]:
pd.read_csv('training_dataset.csv').sort_values('sent_time')['hacker_created_at']

98637     1447248771
86546     1444305615
251159    1423128151
86547     1444742421
68455     1440755089
155565    1383607184
280178    1451802397
198687    1407866817
149744    1353778104
291145    1452525985
175903    1401993178
48572     1439020391
107349    1448730498
6945      1432732569
127359    1450374733
224041    1418481175
39269     1437560973
67774     1440781272
279629    1451747981
242498    1423760795
316275    1454661693
53077     1440361710
156347    1385794932
212451    1414922811
310684    1454455716
66545     1442199588
295643    1453478011
279867    1451558216
322361    1455191175
234783    1420459135
             ...    
430991    1459891398
244147    1424974074
453972    1460870468
255018    1428048843
212038    1414538529
475319    1462140721
482347    1462835625
328163    1455550843
467031    1461559298
371523    1457728775
484341    1462848711
475587    1462163737
266189    1429512689
469626    1461711387
134009    1450848592
483714    1462913791
473413    146